# 2 - Fluid particles

Topics covered in this tutorial:

- basic functionalities of [ParticlesSPH](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#base-modules) class
- drawing markers: random vs. regular "tesselation"
- initializing velocities as $\mathbf v(0) = \mathbf u(\mathbf x(0))$ via a [GenericFluidEquilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#generic-fluid-equilibria)
- velocity push with [PushVinEfield](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinEfield)
- velocity push with [PushVinSPHpressure](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinSPHpressure)

## Fluid flow in external force field

Let $\Omega \subset \mathbb R^3$ be a box (cuboid). We search for trajectories $(\mathbf x_p, \mathbf v_p): [0,T] \to \Omega \times \mathbb R^3$, $p = 0, \ldots, N-1$ that satisfy

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= -\nabla p(\mathbf x_p) \qquad && \mathbf v_p(0) = \mathbf u(\mathbf x_p(0))\,,
 \end{align}
$$

where $p \in H^1(\Omega)$ is some given function.
In Struphy, the position coordinates are updated in logical space $[0, 1]^3 = F^{-1}(\Omega)$, for instance with the Propagator [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) which we shall use in what follows.

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -.5
r1 = .5
l2 = -.5
r2 = .5
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
# define the initial flow

from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np

def u_fun(x, y, z):
    ux = -np.cos(np.pi*x)*np.sin(np.pi*y)
    uy = np.sin(np.pi*x)*np.cos(np.pi*y)
    uz = 0 * x 
    return ux, uy, uz

p_fun = lambda x, y, z: 0.5*(np.sin(np.pi*x)**2 + np.sin(np.pi*y)**2)
n_fun = lambda x, y, z: 1. + 0*x

bel_flow = GenericCartesianFluidEquilibrium(u_xyz=u_fun, p_xyz=p_fun, n_xyz=n_fun)
bel_flow.domain = domain
p_xyz = bel_flow.p_xyz
p0 = bel_flow.p0

In [ ]:
from struphy.pic.particles import ParticlesSPH

# particle boundary conditions
bc = ['reflect', 'reflect', 'periodic']

# instantiate Particle object (for random drawing of markers)
Np = 1000

particles_1 = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bel_flow,
        Np=Np,
    )

# instantiate Particle object (for regular tesselation drawing of markers)
ppb = 4
boxes_per_dim = (16, 16, 1)
loading = "tesselation"
loading_params = {"n_quad": 1}
bufsize = 0.5

particles_2 = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bel_flow,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        loading=loading,
        loading_params=loading_params,
        bufsize=bufsize,
    )

In [ ]:
particles_1.draw_markers(sort=False)
particles_2.draw_markers(sort=False)

particles_1.initialize_weights()
particles_2.initialize_weights()

In [ ]:
print(f'{particles_1.positions.shape = }')
print(f'{particles_2.positions.shape = }')

In [ ]:
# positions on the physical domain Omega
print(f'random: \n{domain(particles_1.positions).T[:10]}')
print(f'\ntesselation: \n{domain(particles_2.positions).T[:10]}')

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=False)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta_1 = PushEta(particles_1, algo = "forward_euler")
prop_eta_2 = PushEta(particles_2, algo = "forward_euler")

In [ ]:
from struphy.feec.psydac_derham import Derham

Nel = [64, 64, 1]  # Number of grid cells
p = [3, 3, 1]  # spline degrees
spl_kind = [False, False, True]   # spline types (clamped vs. periodic)

derham = Derham(Nel, p, spl_kind)

In [ ]:
p_coeffs = derham.P["0"](p0)
p_coeffs

In [ ]:
from struphy.propagators.propagators_markers import PushVinEfield

# instantiate Propagator object
PushVinEfield.domain = domain
PushVinEfield.derham = derham

In [ ]:
p_h = derham.create_spline_function('pressure', 'H1', coeffs=p_coeffs)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
x = np.linspace(-.5, .5, 100)
y = np.linspace(-.5, .5, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)

plt.subplot(2, 2, 1)
plt.pcolor(xx, yy, p_xyz(xx, yy, 0))
plt.axis('square')
plt.title('p_xyz')
plt.colorbar()

plt.subplot(2, 2, 2)
p_vals = p0(eta1, eta2, 0, squeeze_out=True).T
plt.pcolor(eta1, eta2, p_vals)
plt.axis('square')
plt.title('p logical')
plt.colorbar()

plt.subplot(2, 2, 3)
p_h_vals = p_h(eta1, eta2, 0, squeeze_out=True).T
plt.pcolor(eta1, eta2, p_h_vals)
plt.axis('square')
plt.title('p_h (logical)')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.pcolor(eta1, eta2, np.abs(p_vals - p_h_vals))
plt.axis('square')
plt.title('difference')
plt.colorbar()

In [ ]:
grad_p = derham.grad.dot(p_coeffs)
grad_p.update_ghost_regions() # very important, we will move it inside grad
grad_p *= -1.
prop_v_1 = PushVinEfield(particles_1, e_field=grad_p)
prop_v_2 = PushVinEfield(particles_2, e_field=grad_p)

In [ ]:
fig = plt.figure(figsize=(15, 8))
ax1 = fig.add_subplot(1, 2, 1, projection="3d")
pos_1 = domain(particles_1.positions).T
ax1.scatter(pos_1[:, 0], pos_1[:, 1], pos_1[:, 2])
ax1.set_title("random starting positions")

ax2 = fig.add_subplot(1, 2, 2, projection="3d")
pos_2 = domain(particles_2.positions).T
ax2.scatter(pos_2[:, 0],pos_2[:, 1],pos_2[:, 2])
ax2.set_title("starting positions from tesselation")

In [ ]:
import numpy as np

# time stepping
dt = 0.02
Nt = 200

# random particles
pos_1 = np.zeros((Nt + 1, particles_1.Np, 3), dtype=float)
velo_1 = np.zeros((Nt + 1, particles_1.Np, 3), dtype=float)
energy_1 = np.zeros((Nt + 1, particles_1.Np), dtype=float)

# particles_1.draw_markers(sort=False)
# particles_1.initialize_weights()

pos_1[0] = domain(particles_1.positions).T
velo_1[0] = particles_1.velocities
energy_1[0] = .5*(velo_1[0, : , 0]**2 + velo_1[0, : , 1]**2) + p_h(particles_1.positions)

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta_1(dt/2)
    prop_v_1(dt)
    prop_eta_1(dt/2)
    
    # positions on the physical domain Omega
    pos_1[n] = domain(particles_1.positions).T
    velo_1[n] = particles_1.velocities
    
    energy_1[n] = .5*(velo_1[n, : , 0]**2 + velo_1[n, : , 1]**2) + p_h(particles_1.positions)

In [ ]:
# energy plots (random)
fig = plt.figure(figsize = (13, 6))

plt.subplot(2, 2, 1)
plt.plot(time_vec, energy_1[:, 0])
plt.title('particle 1')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 2)
plt.plot(time_vec, energy_1[:, 1])
plt.title('particle 2')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 3)
plt.plot(time_vec, energy_1[:, 2])
plt.title('particle 3')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 4)
plt.plot(time_vec, energy_1[:, 3])
plt.title('particle 4')
plt.xlabel('time')
plt.ylabel('energy')

In [ ]:
plt.figure(figsize=(12, 28))

coloring = np.select([pos_1[0,:,0]<=-0.2, np.abs(pos_1[0,:,0]) < +0.2, pos_1[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])

interval = Nt/20
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(5, 2, plot_ct)
        ax = plt.gca() 
        plt.scatter(pos_1[i, :, 0], pos_1[i, :, 1], c=coloring)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 10:
        break

In [ ]:
# regular tesselation particles

pos_2 = np.zeros((Nt + 1, particles_2.Np, 3), dtype=float)
velo_2 = np.zeros((Nt + 1, particles_2.Np, 3), dtype=float)
energy_2 = np.zeros((Nt + 1, particles_2.Np), dtype=float)

# particles_1.draw_markers(sort=False)
# particles_1.initialize_weights()

pos_2[0] = domain(particles_2.positions).T
velo_2[0] = particles_2.velocities
energy_2[0] = .5*(velo_2[0, : , 0]**2 + velo_2[0, : , 1]**2) + p_h(particles_2.positions)

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta_2(dt/2)
    prop_v_2(dt)
    prop_eta_2(dt/2)
    
    # positions on the physical domain Omega
    pos_2[n] = domain(particles_2.positions).T
    velo_2[n] = particles_2.velocities
    
    energy_2[n] = .5*(velo_2[n, : , 0]**2 + velo_2[n, : , 1]**2) + p_h(particles_2.positions)

In [ ]:
# energy plots (tesselation)
fig = plt.figure(figsize = (13, 6))

plt.subplot(2, 2, 1)
plt.plot(time_vec, energy_2[:, 0])
plt.title('particle 1')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 2)
plt.plot(time_vec, energy_2[:, 1])
plt.title('particle 2')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 3)
plt.plot(time_vec, energy_2[:, 2])
plt.title('particle 3')
plt.xlabel('time')
plt.ylabel('energy')

plt.subplot(2, 2, 4)
plt.plot(time_vec, energy_2[:, 3])
plt.title('particle 4')
plt.xlabel('time')
plt.ylabel('energy')

In [ ]:
plt.figure(figsize=(12, 28))

coloring = np.select([pos_2[0,:,0]<=-0.2, np.abs(pos_2[0,:,0]) < +0.2, pos_2[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])

interval = Nt/20
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(5, 2, plot_ct)
        ax = plt.gca() 
        plt.scatter(pos_2[i, :, 0], pos_2[i, :, 1], c=coloring)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 10:
        break

In [ ]:
make_movie = False
if make_movie:
    import matplotlib.animation as animation
    n_frame = Nt
    fig, axs = plt.subplots(1, 2, figsize=(12, 8))

    coloring_1 = np.select([pos_1[0,:,0]<=-0.2, np.abs(pos_1[0,:,0]) < +0.2, pos_1[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])
    scat_1 = axs[0].scatter(pos_1[0,:,0], pos_1[0,:,1], c=coloring_1)
    axs[0].set_xlim([-0.5,0.5])
    axs[0].set_ylim([-0.5,0.5])
    axs[0].set_aspect('equal')
    
    coloring_2 = np.select([pos_2[0,:,0]<=-0.2, np.abs(pos_2[0,:,0]) < +0.2, pos_2[0,:,0] >= 0.2],
                        [-1.0, 0.0, +1.0])
    scat_2 = axs[1].scatter(pos_2[0,:,0], pos_2[0,:,1], c=coloring_2)
    axs[1].set_xlim([-0.5,0.5])
    axs[1].set_ylim([-0.5,0.5])
    axs[1].set_aspect('equal')

    f = lambda x, y: np.cos(np.pi*x)*np.cos(np.pi*y)
    axs[0].contour(xx, yy, f(xx, yy))
    axs[0].set_title(f'time = {time_vec[0]:4.2f}')
    axs[1].contour(xx, yy, f(xx, yy))
    axs[1].set_title(f'time = {time_vec[0]:4.2f}')

    def update_frame(frame):
        scat_1.set_offsets(pos_1[frame,:,:2])
        axs[0].set_title(f'time = {time_vec[frame]:4.2f}')
        
        scat_2.set_offsets(pos_2[frame,:,:2])
        axs[1].set_title(f'time = {time_vec[frame]:4.2f}')
        return scat_1, scat_2

    ani = animation.FuncAnimation(fig=fig, func=update_frame, frames = n_frame)
    ani.save("tutorial_02_movie.gif")

## Sound wave

We use SPH to solve Euler's equations

$$
\begin{align}
 \partial_t \rho + \nabla \cdot (\rho \mathbf u) &= 0\,,
 \\[2mm]
 \rho(\partial_t \mathbf u + \mathbf u \cdot \nabla \mathbf u) &= - \nabla \left(\rho^2 \frac{\partial \mathcal U(\rho, S)}{\partial \rho} \right)\,,
 \\[2mm]
 \partial_t S + \mathbf u \cdot \nabla S &= 0\,,
 \end{align}
$$

where $S$ denotes the entropy per unit mass and the internal energy per unit mass is 

$$
\mathcal U(\rho, S) = \kappa(S) \log \rho\,.
$$

The SPH discretization leads to ODEs for $N$ particles indexed by $p$,

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= -\kappa_{p}(0) \sum_{i=1}^N w_i \left(\frac{1}{\rho^{N,h}(\mathbf x_p)} + \frac{1}{\rho^{N,h}(\mathbf x_i)} \right) \nabla W_h(\mathbf x_p - \mathbf x_i) \qquad && \mathbf v_p(0) = \mathbf u(\mathbf x_p(0))\,,
 \end{align}
$$

where the smoothed density reads

$$
 \rho^{N,h}(\mathbf x) = \sum_{j=1}^N w_j W_h(\mathbf x - \mathbf x_j)\,,
$$

with weights $w_p = const.$ and where $W_h(\mathbf x)$ is a suitable smoothing kernel.
The velocity update is performed with the Propagator [PushVinSPHpressure](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVinSPHpressure).

We shall compute:
* a standing sound wave in 1d (linear dynamics)
* a gas expansion in 2d (nonlinear example).

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = 0
r1 = 2.5
l2 = 0
r2 = 1.
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
cst_vel = {"ux": 0., 
           "uy": 0.,
           "uz": 0.,
           "density_profile": "constant"}
bckgr_params = {"ConstantVelocity": cst_vel}

mode_params = {"given_in_basis": "0",
               "ls": [1],
               "amps": [1e-2]}
modes = {"ModesSin": mode_params}
pert_params = {"n": modes}

In [ ]:
#particle initialization 
from struphy.pic.particles import ParticlesSPH

# marker parameters
ppb = 16
nx = 16
ny = 1
nz = 1
boxes_per_dim = (nx, ny, nz)
bc = ['periodic']*3
loading = "tesselation"
loading_params = {"n_quad": 1}

# instantiate Particle object
particles = ParticlesSPH(
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        bc=bc,
        domain=domain,
        bckgr_params=bckgr_params,
        pert_params=pert_params,
        loading=loading,
        loading_params=loading_params,
        verbose=False,
        bufsize=0.5,
        n_cols_aux=3,
    )

In [ ]:
particles.draw_markers(sort=False)
particles.initialize_weights()

In [ ]:
particles.markers.shape

In [ ]:
particles.sorting_boxes.boxes.shape

In [ ]:
import numpy as np

np.set_printoptions(suppress=True,linewidth=300,threshold=300,formatter=dict(float=lambda x: "%.5f" % x))

plot_pts = 32

components = [True, False, False, False, False, False]
nx_b = plot_pts
be_x = np.linspace(0, 1, nx_b + 1)
bin_edges = [be_x]
f_bin, df_bin = particles.binning(components, bin_edges, divide_by_jac=False)
f_bin.shape

In [ ]:
x = np.linspace(l1, r1, 100)
eta1 = np.linspace(0, 1, plot_pts)
eta2 = np.linspace(0, 1, 1)
eta3 = np.linspace(0, 1, 1)
ee1, ee2, ee3 = np.meshgrid(eta1, eta2, eta3, indexing='ij')

In [ ]:
kernel_type = "gaussian_1d" 
h1 = 1/nx
h2 = 1/ny
h3 = 1/nz

n_sph_init = particles.eval_density(ee1, ee2, ee3, h1=h1, h2=h2, h3=h3, kernel_type=kernel_type, fast=True,)
n_sph_init.shape

In [ ]:
logpos = particles.positions
weights = particles.weights
print(f'{logpos.shape = }')
print(f'{weights.shape = }')

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(10, 10))

n0 = particles.f_init

plt.subplot(2, 2, 1)
plt.plot(eta1, np.squeeze(n0(eta1, eta2, eta3).T))
plt.title('$n/\sqrt{g}$ (0-form)')

plt.subplot(2, 2, 2)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
ax.set_yticks(np.linspace(0, 1, ny + 1))
plt.tick_params(labelbottom = False) 
coloring = weights
plt.scatter(logpos[:, 0], logpos[:, 1], c=coloring, s=.25)
plt.grid(c='k')
plt.axis('square')
plt.title('n0_scatter')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.colorbar()

plt.subplot(2, 2, 3)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
#ax.set_yticks(np.linspace(0, 1., ny + 1))
plt.tick_params(labelbottom = False) 
plt.plot(eta1, n_sph_init[:, 0, 0])
plt.grid()
plt.title(f'n_sph_init')

plt.subplot(2, 2, 4)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
#ax.set_yticks(np.linspace(0, 1., ny + 1))
plt.tick_params(labelbottom = False) 
bc_x = (be_x[:-1] + be_x[1:]) / 2. # centers of binning cells
plt.plot(bc_x, df_bin.T)
#plt.grid()
plt.title(f'n_binned')

In [ ]:
from struphy.pic.sph_smoothing_kernels import linear_uni, trigonometric_uni, gaussian_uni

x = np.linspace(-1, 1, 200)
out1 = np.zeros_like(x)
out2 = np.zeros_like(x)
out3 = np.zeros_like(x)

for i, xi in enumerate(x):
    out1[i] = trigonometric_uni(xi, 1.)
    out2[i] = gaussian_uni(xi, 1.)
    out3[i] = linear_uni(xi, 1.)
plt.plot(x, out1, label="trigonometric")
plt.plot(x, out2, label="gaussian")
plt.plot(x, out3, label = "linear")
plt.title('Some smoothing kernels')
plt.legend()

In [ ]:
from struphy.propagators.propagators_markers import PushEta, PushVinSPHpressure

PushEta.domain = domain
prop_eta = PushEta(particles, algo = "forward_euler")

PushVinSPHpressure.domain = domain
algo = "forward_euler"
kernel_width = (h1, h2, h3)
prop_v = PushVinSPHpressure(particles,
                            kernel_type = kernel_type,
                            kernel_width = kernel_width, 
                            algo = algo)

In [ ]:
import numpy as np

# time stepping
end_time = (r1 - l1) # so that the waves traverse the domain once (c_s = 1)
dt = 0.05*(8/nx) * end_time
Nt = int(end_time/dt)

Np = particles.positions.shape[0]

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
weights = np.zeros((Nt + 1, Np), dtype=float)
n_sph = np.zeros((Nt + 1, *ee1.shape), dtype=float)

pos[0] = domain(particles.positions).T
weights[0] = particles.weights
n_sph[0] = n_sph_init

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0

if True:
    while n < Nt:
        time += dt
        n += 1
        time_vec[n] = time
        
        # advance in time
        prop_eta(dt/2)
        prop_v(dt)
        prop_eta(dt/2)
        
        # positions on the physical domain Omega
        pos[n] = domain(particles.positions).T
        weights[n] = particles.weights
        n_sph[n] = particles.eval_density(ee1, ee2, ee3, h1=h1, h2=h2, h3=h3, kernel_type=kernel_type, fast=True,)

        print(f'{n} time steps done.')

In [ ]:
from matplotlib.ticker import FormatStrFormatter

x, y, z = domain(eta1, eta2, eta3, squeeze_out=True)

plt.figure(figsize=(10, 8))
interval = Nt/10
plot_ct = 0
for i in range(0, Nt + 1):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        ax = plt.gca() 
        
        if plot_ct <= 6:
            style = '-'
        else:
            style = '.'
        plt.plot(x, n_sph[i, :, 0, 0], style, label=f'time={i*dt:4.2f}')
        plt.xlim(l1, r1)
        plt.legend()
        ax.set_xticks(np.linspace(l1, r1, nx + 1))
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        plt.grid(c='k')
        plt.xlabel("x")
        plt.ylabel(r"$\rho$")
        
        plt.title(f'standing sound wave ($c_s = 1$) for {nx = } and {ppb = }')
    if plot_ct == 11:
        break

## Gas expansion

We use the same SPH discretization of Euler's equations as described above for sound waves. However, now we simulate a nonlinear gas expansion.

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -3
r1 = 3
l2 = -3
r2 = 3
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np
T_h = 0.2
gamma = 5/3
n_fun = lambda x, y, z: np.exp(-(x**2 + y**2)/T_h) / 35

bckgr = GenericCartesianFluidEquilibrium(n_xyz=n_fun)
bckgr.domain = domain

In [ ]:
#particle initialization 
from struphy.pic.particles import ParticlesSPH

# marker parameters
ppb = 400
nx = 16
ny = 16
nz = 1
boxes_per_dim = (nx, ny, nz)
bc = ['periodic']*3

# instantiate Particle object (for random drawing of markers)
particles_1 = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
    )

# instantiate Particle object (for regular tesselation drawing of markers)
loading = "tesselation"
loading_params = {"n_quad": 1}
bufsize = 0.5

particles_2 = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        loading=loading,
        loading_params=loading_params,
        bufsize=bufsize,
    )

In [ ]:
particles_1.draw_markers(sort=False)
particles_2.draw_markers(sort=False)

threshold = 1e-1
particles_1.initialize_weights(reject_weights=True, threshold=threshold)
particles_2.initialize_weights(reject_weights=True, threshold=threshold)

In [ ]:
particles_1.weights[:10]

In [ ]:
particles_2.weights[:10]

In [ ]:
print(f'{particles_1.markers.shape = }')
print(f'{particles_2.markers.shape = }')

In [ ]:
print(f'{particles_1.sorting_boxes.boxes.shape = }')
print(f'{particles_2.sorting_boxes.boxes.shape = }')

In [ ]:
components = [True, True, False, False, False, False]
nx_b = 64
ny_b = 64
be_x = np.linspace(0, 1, nx_b + 1)
be_y = np.linspace(0, 1, ny_b + 1)
bin_edges = [be_x, be_y]

f_bin_1, df_bin_1 = particles_1.binning(components, bin_edges, divide_by_jac=False)
f_bin_2, df_bin_2 = particles_2.binning(components, bin_edges, divide_by_jac=False)

In [ ]:
x = np.linspace(l1, r1, 100)
y = np.linspace(l2, r2, 90)
xx, yy = np.meshgrid(x, y, indexing="ij")
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)
eta3 = np.linspace(0,1,1)
ee1, ee2, ee3 = np.meshgrid(eta1, eta2, eta3, indexing="ij")

In [ ]:
kernel_type = "gaussian_2d" 
h1 = 1/nx
h2 = 1/ny
h3 = 1/nz

n_sph_1 = particles_1.eval_density(ee1, ee2, ee3, h1=h1, h2=h2, h3=h3, kernel_type=kernel_type, fast=True,)
n_sph_2 = particles_2.eval_density(ee1, ee2, ee3, h1=h1, h2=h2, h3=h3, kernel_type=kernel_type, fast=True,)

In [ ]:
logpos_1 = particles_1.positions
logpos_2 = particles_2.positions

weights_1 = particles_1.weights
weights_2 = particles_2.weights

print(f'{logpos_1.shape = }')
print(f'{logpos_2.shape = }')
print(f'{weights_1.shape = }')
print(f'{weights_2.shape = }')

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(12, 22))

n_xyz = bckgr.n_xyz
n3 = bckgr.n3

plt.subplot(4, 2, 1)
plt.pcolor(xx, yy, n_fun(xx, yy, 0))
plt.axis('square')
plt.title('n_xyz')
plt.colorbar()

plt.subplot(4, 2, 2)
plt.pcolor(eta1, eta2, n3(eta1,eta2,0, squeeze_out=True).T)
plt.axis('square')
plt.title('$\hat{n}^{\t{vol}}$ (volume form)')
plt.colorbar()

make_scatter = True
if make_scatter:
    plt.subplot(4, 2, 3)
    ax = plt.gca()
    ax.set_xticks(np.linspace(0, 1, nx + 1))
    ax.set_yticks(np.linspace(0, 1., ny + 1))
    plt.tick_params(labelbottom = False) 
    coloring = weights_1
    plt.scatter(logpos_1[:, 0], logpos_1[:, 1], c=coloring, s=.25)
    plt.grid(c='k')
    plt.axis('square')
    plt.title('$\hat{n}^{\t{vol}}$ scatter (random)')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.colorbar()
    
    plt.subplot(4, 2, 4)
    ax = plt.gca()
    ax.set_xticks(np.linspace(0, 1, nx + 1))
    ax.set_yticks(np.linspace(0, 1., ny + 1))
    plt.tick_params(labelbottom = False) 
    coloring = weights_2
    plt.scatter(logpos_2[:, 0], logpos_2[:, 1], c=coloring, s=.25)
    plt.grid(c='k')
    plt.axis('square')
    plt.title('$\hat{n}^{\t{vol}}$ scatter (tesselation)')
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.colorbar()

plt.subplot(4, 2, 5)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
ax.set_yticks(np.linspace(0, 1., ny + 1))
plt.tick_params(labelbottom = False) 
plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sph_1[:,:,0])
plt.grid()
plt.axis('square')
plt.title(f'n_sph (random)')
plt.colorbar()

plt.subplot(4, 2, 6)
ax = plt.gca()
ax.set_xticks(np.linspace(0, 1, nx + 1))
ax.set_yticks(np.linspace(0, 1., ny + 1))
plt.tick_params(labelbottom = False) 
plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sph_2[:,:,0])
plt.grid()
plt.axis('square')
plt.title(f'n_sph (tesselation)')
plt.colorbar()

plt.subplot(4, 2, 7)
ax = plt.gca()
# ax.set_xticks(np.linspace(0, 1, nx + 1))
# ax.set_yticks(np.linspace(0, 1., ny + 1))
# plt.tick_params(labelbottom = False) 
bc_x = (be_x[:-1] + be_x[1:]) / 2. # centers of binning cells
bc_y = (be_y[:-1] + be_y[1:]) / 2.
plt.pcolor(bc_x, bc_y, f_bin_1)
#plt.grid()
plt.axis('square')
plt.title(f'n_binned (random)')
plt.colorbar()

plt.subplot(4, 2, 8)
ax = plt.gca()
# ax.set_xticks(np.linspace(0, 1, nx + 1))
# ax.set_yticks(np.linspace(0, 1., ny + 1))
# plt.tick_params(labelbottom = False) 
bc_x = (be_x[:-1] + be_x[1:]) / 2. # centers of binning cells
bc_y = (be_y[:-1] + be_y[1:]) / 2.
plt.pcolor(bc_x, bc_y, f_bin_2)
#plt.grid()
plt.axis('square')
plt.title(f'n_binned (tesselation)')
plt.colorbar()

In [ ]:
from struphy.pic.sph_smoothing_kernels import linear_uni, trigonometric_uni, gaussian_uni

x = np.linspace(-1, 1, 200)
out1 = np.zeros_like(x)
out2 = np.zeros_like(x)
out3 = np.zeros_like(x)

for i, xi in enumerate(x):
    out1[i] = trigonometric_uni(xi, 1.)
    out2[i] = gaussian_uni(xi, 1.)
    out3[i] = linear_uni(xi, 1.)
plt.plot(x, out1, label="trigonometric")
plt.plot(x, out2, label="gaussian")
plt.plot(x, out3, label = "linear")
plt.title('Some smoothing kernels')
plt.legend()

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=False)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta_1 = PushEta(particles_1, algo = "forward_euler")
prop_eta_2 = PushEta(particles_2, algo = "forward_euler")

In [ ]:
from struphy.propagators.propagators_markers import PushVinSPHpressure

# default parameters of Propagator
opts_sph = PushVinSPHpressure.options(default=False)
print(opts_sph)

In [ ]:
# pass simulation parameters to Propagator class
PushVinSPHpressure.domain = domain

In [ ]:
# instantiate Propagator object
algo = "forward_euler"
kernel_width = (h1, h2, h3)

prop_v_1 = PushVinSPHpressure(particles_1,
                            kernel_type = kernel_type,
                            kernel_width = kernel_width, 
                            algo = algo)

prop_v_2 = PushVinSPHpressure(particles_2,
                            kernel_type = kernel_type,
                            kernel_width = kernel_width, 
                            algo = algo)

In [ ]:
import numpy as np

# time stepping
dt = 0.04
Nt = 40

Np = particles_1.positions.shape[0]
pos_1 = np.zeros((Nt + 1, Np, 3), dtype=float)
velo_1 = np.zeros((Nt + 1, Np, 3), dtype=float)
energy_1 = np.zeros((Nt + 1, Np), dtype=float)

pos_1[0] = domain(particles_1.positions).T
velo_1[0] = particles_1.velocities

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta_1(dt/2)
    prop_v_1(dt)
    prop_eta_1(dt/2)
    
    # positions on the physical domain Omega
    pos_1[n] = domain(particles_1.positions).T
    velo_1[n] = particles_1.velocities
    
    print(f'{n} time steps done.')

In [ ]:
plt.figure(figsize=(12, 24))
interval = Nt/10
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(4, 2, plot_ct)
        ax = plt.gca() 
        coloring = weights_1
        plt.scatter(pos_1[i, :, 0], pos_1[i, :, 1], c=coloring, s=.25)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 8:
        break

In [ ]:
Np = particles_2.positions.shape[0]
pos_2 = np.zeros((Nt + 1, Np, 3), dtype=float)
velo_2 = np.zeros((Nt + 1, Np, 3), dtype=float)
energy_2 = np.zeros((Nt + 1, Np), dtype=float)

pos_2[0] = domain(particles_2.positions).T
velo_2[0] = particles_2.velocities

time = 0.
time_vec = np.zeros(Nt + 1, dtype=float)
n = 0
while n < Nt:
    time += dt
    n += 1
    time_vec[n] = time
    
    # advance in time
    prop_eta_2(dt/2)
    prop_v_2(dt)
    prop_eta_2(dt/2)
    
    # positions on the physical domain Omega
    pos_2[n] = domain(particles_2.positions).T
    velo_2[n] = particles_2.velocities
    
    print(f'{n} time steps done.')

In [ ]:
plt.figure(figsize=(12, 24))
interval = Nt/10
plot_ct = 0
for i in range(Nt):
    if i % interval == 0:
        print(f'{i = }')
        plot_ct += 1
        plt.subplot(4, 2, plot_ct)
        ax = plt.gca() 
        coloring = weights_2
        plt.scatter(pos_2[i, :, 0], pos_2[i, :, 1], c=coloring, s=.25)
        plt.axis('square')
        plt.title('n0_scatter')
        plt.xlim(l1, r1)
        plt.ylim(l2, r2)
        plt.colorbar()
        plt.title(f'Gas at t={i*dt}')
    if plot_ct == 8:
        break